In [2]:
import os

In [3]:
%pwd

'c:\\Users\\adamv\\Desktop\\MLL\\research'

In [4]:
os.chdir("../")

In [5]:
%pwd

'c:\\Users\\adamv\\Desktop\\MLL'

In [6]:
import dagshub
dagshub.init(repo_owner='adam.vamshikrishna', repo_name='MLflow', mlflow=True)

Initialized MLflow to track repo "adam.vamshikrishna/MLflow"

[2024-05-31 14:36:19,962: INFO: helpers: Initialized MLflow to track repo "adam.vamshikrishna/MLflow"]


Repository adam.vamshikrishna/MLflow initialized!

[2024-05-31 14:36:19,965: INFO: helpers: Repository adam.vamshikrishna/MLflow initialized!]


In [7]:
import tensorflow as tf

In [11]:
model = tf.keras.models.load_model("artifacts/training/model.h5") 

In [8]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class EvaluationConfig:
    path_of_model: Path
    training_data: Path
    all_params: dict
    mlflow_uri: str
    params_image_size: list
    params_batch_size: int

In [9]:
from cnnClassifer.constants import *
from cnnClassifer.utils.Common import read_yaml, create_directories, save_json

In [10]:
class ConfigurationManager:
    def __init__(
        self, 
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        create_directories([self.config.artifacts_root])

    
    def get_evaluation_config(self) -> EvaluationConfig:
        eval_config = EvaluationConfig(
            path_of_model="artifacts/training/model.h5",
            training_data="artifacts/data_ingestion/kidney-ct-scan-image",
            mlflow_uri="https://dagshub.com/adam.vamshikrishna/MLflow.mlflow/",
            all_params=self.params,
            params_image_size=self.params.IMAGE_SIZE,
            params_batch_size=self.params.BATCH_SIZE
        )
        return eval_config

In [12]:
import tensorflow as tf
from pathlib import Path
import mlflow
import mlflow.keras
from urllib.parse import urlparse
from typing import Dict, Any, Tuple

In [13]:
from pathlib import Path
import tensorflow as tf
import mlflow
from urllib.parse import urlparse



class Evaluation:
    def __init__(self, config: EvaluationConfig):
        self.config = config
        self.model = None
        self.score = None
        self.valid_generator = None

    
    def train_valid_generator(self):
        datagenerator_kwargs: Dict[str, Any] = dict(
            rescale=1./255,          # float
            validation_split=0.30    # float
        )

        dataflow_kwargs: Dict[str, Any]  = dict(
            target_size=self.config.params_image_size[:-1],
            batch_size=self.config.params_batch_size,
            interpolation="bilinear"
        )

        valid_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(
            **datagenerator_kwargs
        )

        self.valid_generator = valid_datagenerator.flow_from_directory(
            directory=self.config.training_data,
            subset="validation",
            shuffle=False,
            **dataflow_kwargs
        )
        



    @staticmethod
    def load_model(path: Path) -> tf.keras.Model:
        return tf.keras.models.load_model(Path)

    def _valid_generator(self):
        # Assuming you have a method to set self.valid_generator
        pass

    def evaluation(self):
        self.model = self.load_model(self.config.path_of_model)
        self._valid_generator()
        self.score = self.model.evaluate(self.valid_generator)
        self.save_score()

    def save_score(self):
        scores = {"loss": self.score[0], "accuracy": self.score[1]}
        save_json(path=Path("scores.json"), data=scores)

    def log_into_mlflow(self):
        mlflow.set_registry_uri(self.config.mlflow_uri)
        tracking_url_type_store = urlparse(mlflow.get_tracking_uri()).scheme

        with mlflow.start_run():
            mlflow.log_params(self.config.all_params)
            mlflow.log_metrics(
                {"loss": self.score[0], "accuracy": self.score[1]}
            )
            # Model registry does not work with file store
            if tracking_url_type_store != "file":
                # Register the model
                # There are other ways to use the Model Registry, which depends on the use case,
                # please refer to the doc for more information:
                # https://mlflow.org/docs/latest/model-registry.html#api-workflow
                mlflow.keras.log_model(self.model, "model", registered_model_name="VGG16Model")
            else:
                mlflow.keras.log_model(self.model, "model")

In [ ]:
try:
    config = ConfigurationManager()
    eval_config = config.get_evaluation_config()
    evaluation = Evaluation(eval_config)
    evaluation.evaluation()
    evaluation.log_into_mlflow()

except Exception as e:
   raise e

In [14]:
import tensorflow as tf
from dataclasses import dataclass
from pathlib import Path
import mlflow
import mlflow.keras
from urllib.parse import urlparse
from typing import Dict, Any
from cnnClassifer.constants import *
from cnnClassifer.utils.Common import read_yaml, create_directories, save_json

@dataclass(frozen=True)
class EvaluationConfig:
    path_of_model: Path
    training_data: Path
    all_params: dict
    mlflow_uri: str
    params_image_size: list
    params_batch_size: int

class ConfigurationManager:
    def __init__(self, config_filepath=CONFIG_FILE_PATH, params_filepath=PARAMS_FILE_PATH):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        create_directories([self.config.artifacts_root])
    
    def get_evaluation_config(self) -> EvaluationConfig:
        eval_config = EvaluationConfig(
            path_of_model=Path("artifacts/training/model.h5"),
            training_data=Path("artifacts/data_ingestion/kidney-ct-scan-image"),
            mlflow_uri="https://dagshub.com/adam.vamshikrishna/MLflow.mlflow/",
            all_params=self.params,
            params_image_size=self.params.IMAGE_SIZE,
            params_batch_size=self.params.BATCH_SIZE
        )
        return eval_config 

class Evaluation:
    def __init__(self, config: EvaluationConfig):
        self.config = config
        self.model = None
        self.score = None
        self.valid_generator = None

    def train_valid_generator(self):
        datagenerator_kwargs: Dict[str, Any] = dict(
            rescale=1./255,
            validation_split=0.30
        )

        dataflow_kwargs: Dict[str, Any] = dict(
            target_size=tuple(self.config.params_image_size[:-1]),
            batch_size=self.config.params_batch_size,
            interpolation="bilinear"
        )

        valid_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(
            **datagenerator_kwargs
        )

        self.valid_generator = valid_datagenerator.flow_from_directory(
            directory=str(self.config.training_data),
            subset="validation",
            shuffle=False,
            **dataflow_kwargs
        )

    @staticmethod
    def load_model(path: Path) -> tf.keras.Model:
        return tf.keras.models.load_model(path)

    def evaluation(self):
        self.model = self.load_model(self.config.path_of_model)
        self.train_valid_generator()
        self.score = self.model.evaluate(self.valid_generator)
        self.save_score()

    def save_score(self):
        scores = {"loss": self.score[0], "accuracy": self.score[1]}
        save_json(path=Path("scores.json"), data=scores)

    def log_into_mlflow(self):
        mlflow.set_registry_uri(self.config.mlflow_uri)
        tracking_url_type_store = urlparse(mlflow.get_tracking_uri()).scheme

        with mlflow.start_run():
            mlflow.log_params(self.config.all_params)
            mlflow.log_metrics(
                {"loss": self.score[0], "accuracy": self.score[1]}
            )
            if tracking_url_type_store != "file":
                mlflow.keras.log_model(self.model, "model", registered_model_name="VGG16Model")
            else:
                mlflow.keras.log_model(self.model, "model")

# Example usage:
try:
    config = ConfigurationManager()
    eval_config = config.get_evaluation_config()
    evaluation = Evaluation(eval_config)
    evaluation.evaluation()
    evaluation.log_into_mlflow()

except Exception as e:
    print(f"An error occurred: {e}")
    raise e

[2024-05-31 14:36:52,162: INFO: Common: yaml file: config\config.yaml loaded successfully]
[2024-05-31 14:36:52,165: INFO: Common: yaml file: params.yaml loaded successfully]
[2024-05-31 14:36:52,166: INFO: Common: created directory at: artifacts]
Found 139 images belonging to 2 classes.
9/9 [==============================] - 13s 1s/step - loss: 9.4451 - accuracy: 0.5036
[2024-05-31 14:37:05,668: INFO: Common: json file saved at: scores.json]


2024/05/31 14:37:06 WARNING mlflow.tensorflow: You are saving a TensorFlow Core model or Keras model without a signature. Inference with mlflow.pyfunc.spark_udf() will not work unless the model's pyfunc representation accepts pandas DataFrames as inference inputs.


[2024-05-31 14:37:07,757: WARNING: save: Found untraced functions such as _jit_compiled_convolution_op, _jit_compiled_convolution_op, _jit_compiled_convolution_op, _jit_compiled_convolution_op, _jit_compiled_convolution_op while saving (showing 5 of 14). These functions will not be directly callable after loading.]
INFO:tensorflow:Assets written to: C:\Users\adamv\AppData\Local\Temp\tmpwa8qoejw\model\data\model\assets
[2024-05-31 14:37:08,372: INFO: builder_impl: Assets written to: C:\Users\adamv\AppData\Local\Temp\tmpwa8qoejw\model\data\model\assets]


c:\Users\adamv\anaconda3\envs\kidney\lib\site-packages\_distutils_hack\__init__.py:26: UserWarning: Setuptools is replacing distutils.
  warnings.warn("Setuptools is replacing distutils.")
Registered model 'VGG16Model' already exists. Creating a new version of this model...
2024/05/31 14:37:31 INFO mlflow.tracking._model_registry.client: Waiting up to 300 seconds for model version to finish creation.                     Model name: VGG16Model, version 2
Created version '2' of model 'VGG16Model'.
